In [487]:
import pandas as pd

data = pd.read_csv("SBUX.csv")
data = data.drop(["Date", "Close"], axis=1)
data = data.astype(int)

In [309]:
data

,Low,Open,Volume,High,Adjusted Close
0,0,0,224358400,0,0
1,0,0,58732800,0,0
2,0,0,34777600,0,0
3,0,0,18316800,0,0
4,0,0,13996800,0,0
...,...,...,...,...,...
7668,101,103,8383000,104,102
7669,101,101,4194300,102,101
7670,102,102,5465100,103,103
7671,101,103,7998900,103,101


In [525]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X = data.drop(columns=["Adjusted Close"])
y = data["Adjusted Close"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.14, random_state = 10)

model = DecisionTreeClassifier(max_depth = 199, min_samples_leaf = 14, min_samples_split = 28)
model.fit(X_train.values, y_train)

# Low, Open, Volume, High

predictions = model.predict([[90, 92, 9257000, 92], [95, 96, 6195200, 97]])
predictions1 = model.predict(X_test)
score = accuracy_score(y_test, predictions1)
print(predictions, f"\nAccuracy: {score}")

[91 90] 
Accuracy: 0.706046511627907


/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(


In [527]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from bayes_opt import BayesianOptimization

# Define the function to optimize using cross-validation
def dtree_cv(max_depth, min_samples_split, min_samples_leaf):
    # Define the model with the parameters to be optimized
    estimator = DecisionTreeRegressor(
        max_depth=int(max_depth),
        min_samples_split=int(min_samples_split),
        min_samples_leaf=int(min_samples_leaf),
        random_state=2
    )
    cval = cross_val_score(estimator, X_train, y_train, scoring='neg_mean_squared_error', cv=5)
    return cval.mean() # The optimizer tries to maximize the function, so we negate the score to minimize it

# Define the parameter bounds
param_bounds = {
    'max_depth': (1, 100),
    'min_samples_split': (2, 100),
    'min_samples_leaf': (1, 100)
}

optimizer = BayesianOptimization(
    f=dtree_cv,
    pbounds=param_bounds,
    random_state=1,
)

optimizer.maximize(n_iter=100, init_points=10) # Bayesian optimization
best_params_bayes = optimizer.max['params']
best_params_bayes['max_depth'] = int(best_params_bayes['max_depth'])
best_params_bayes['min_samples_split'] = int(best_params_bayes['min_samples_split'])
best_params_bayes['min_samples_leaf'] = int(best_params_bayes['min_samples_leaf'])
best_score_bayes = optimizer.max['target']

print(f"Best Parameters (Bayesian Optimization): {best_params_bayes}")
print(f"Best Score (Bayesian Optimization): {best_score_bayes}")

|   iter    |  target   | max_depth | min_sa... | min_sa... |
-------------------------------------------------------------
| 1         | -156.8    | 834.6     | 1.441e+03 | 2.229     |
| 2         | -23.89    | 605.4     | 294.4     | 186.5     |
| 3         | -95.7     | 373.3     | 691.8     | 794.7     |
| 4         | -133.0    | 1.078e+03 | 839.0     | 1.371e+03 |
| 5         | -231.4    | 409.7     | 1.756e+03 | 56.72     |
| 6         | -133.0    | 1.341e+03 | 835.2     | 1.118e+03 |
| 7         | -128.9    | 281.6     | 397.0     | 1.602e+03 |
| 8         | -92.54    | 1.937e+03 | 627.5     | 1.385e+03 |
| 9         | -237.3    | 1.753e+03 | 1.789e+03 | 171.9     |
| 10        | -128.9    | 79.07     | 340.5     | 1.757e+03 |
| 11        | -133.5    | 197.6     | 842.8     | 1.916e+03 |
| 12        | -145.8    | 1.067e+03 | 1.384e+03 | 632.4     |
| 13        | -194.6    | 1.373e+03 | 1.669e+03 | 38.54     |
| 14        | -284.7    | 1.501e+03 | 1.978e+03 | 1.497e+03 |
| 15    

KeyboardInterrupt: 